## Modeling Data / Feature Selection
Combine and save datasets, create set of selected features

Setup

In [2]:
import pandas as pd
from pathlib import Path
import pickle
from sklearn.model_selection import TimeSeriesSplit
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.stattools import acf
import numpy as np

%config InteractiveShell.ast_node_interactivity = 'all'

In [ ]:
def dd_hourly_pivot(df): # requires country actuals
    '''
    Description: pivots table long-ways, with country and aggregated hourly 
    actuals load as columns
    '''
    return (df.pivot_table(index=['day', 'country', 'hdd', 'cdd'], 
                           values='load_actual', 
                           columns = 'hour', 
                           aggfunc='sum')
                            # remove first day with NaNs
              .dropna()
              .reset_index(level=1))

def dd_plot_norm_avg_hourly(df, group, pivot=True): # requires df from hourly_pivot function
    '''
    Description: min-max normalization of input data across each hour
    '''
    df = df.groupby(by=group, as_index=True).mean(numeric_only=True).round(2) # .reset_index()
    # hour columns
    cols = list(range(24))

    # columns for row-wise min-max normalization
    df['min'] = df[cols].min(axis=1)
    df['max'] = df[cols].max(axis=1)

    # Row-wise Min-max normalization 
    df[cols] = (df[cols].sub(df['min'], axis=0)
                        .div(df['max'] - df['min'], axis=0))
    
    # Melt for overlapping lineplots
    if pivot:
        df = (df.reset_index()
                .melt(id_vars=group,
                      value_vars=cols)
                .assign(hour = lambda x: pd.to_numeric(x.hour))
                )
        
    return df

def multi_lineplots(plot_df, group): # requires df from norm_avg_hourly function  
    '''
    Description: Creates overlapping line plots for each country given a dataframe
    from the 'norm_avg_hourly' function.
    '''
    colors = sns.color_palette('magma')
    _ = plt.figure(figsize=(6, 4))

    # Create dataset and plot for each time series in group
    for grouping, color in zip(plot_df[group].unique(), colors):
        line_plot = (plot_df[plot_df[group] == grouping])
        _ = sns.lineplot(
            data=line_plot,
            x='hour',
            y='value',
            color=color,
            label = str(grouping)
        )

        _ = plt.xlabel('Hour')
        _ = plt.ylabel('Normalized Load Actual')
        _ = plt.title(f'Avg Load Shapes by {group} 2015-2019')
        plt.show()

def lags_and_rolling_means(df, list_vars=None, group='country', list_hours=1):
    '''
    Description: With data frame, list of variables, and hours to lag/roll, 
    adds lags and rolling mean to a copy of the dataframe. 
    *Requires a group argument
    '''
    df_copy = df.copy(deep=True)

    # iterate over variables
    for var in list_vars:
        # iterate over hours
        for hour in list_hours:
            # lag columns
            lag_col = f"{var}_lag{int(hour/24)}_days" 
            df_copy[lag_col] = df_copy.groupby(group, as_index=False)[var].shift(hour)
            # rolling columns
            roll_col = f"{var}_roll{int(hour/24)}_days" 
            df_copy[roll_col] = df_copy.groupby(group, as_index=False)[var].rolling(hour).mean()[[var]]
    
    return df_copy

Read in Data

In [165]:
# Filepaths
cd = Path.cwd()
data_dir = str(cd.parents[1])
load_wthr_dt = data_dir + '/datasets/country_energy/pivot_country_weather.pickle'
load_energy_dt = data_dir + '/datasets/country_energy/country_load.pickle'

# Read pickle data
with open(load_wthr_dt, 'rb') as f:
    wthr_dt = (pickle.load(f).loc[lambda x: x.utc_timestamp.between('2015-01-01', '2019-04-30')]
                             # set_index introduces duplicate rows ***** 227526 without set_index, 227339 with
                             .set_index(['utc_timestamp']) 
                             # simplify var names
                             .rename(columns={'radiation_direct_horizontal': 'radi_direct',
                                              'radiation_diffuse_horizontal': 'radi_diffuse'}))

with open(load_energy_dt, 'rb') as f:
    load_dt = (pickle.load(f).loc[lambda x: x.utc_timestamp.between('2015-01-01', '2019-04-30')]
                             .set_index(['utc_timestamp']) # set_index introduces duplicate rows *****
                             # simplify var names
                             .rename(columns={'load_actual_entsoe_transparency': 'load_actual'}))

# View data
print(f'Weather data')
wthr_dt.head(2)
wthr_dt.tail(2)
print(f'\nLoad data')
load_dt.head(2)
load_dt.tail(2)
print(f'\nnrow weather data: {len(wthr_dt)} nrow load data: {len(load_dt)}')

len(wthr_dt)
len(wthr_dt.reset_index().drop_duplicates()) * 4
len(load_dt)
len(load_dt.reset_index().drop_duplicates())

Weather data


,country,temperature,radi_direct,radi_diffuse,day,mean_temp,max_temp,min_temp,hdd,cdd
utc_timestamp,,,,,,,,,,
2015-01-01 00:00:00+00:00,LU,-1.983,0.0,0.0,2015-01-01,-1.119535,3.298,-8.399,1,0
2015-01-01 01:00:00+00:00,LU,-2.083,0.0,0.0,2015-01-01,-1.119535,3.298,-8.399,1,0


,country,temperature,radi_direct,radi_diffuse,day,mean_temp,max_temp,min_temp,hdd,cdd
utc_timestamp,,,,,,,,,,
2019-04-29 23:00:00+00:00,AT,3.977,0.0,0.0,2019-04-29,9.270035,16.482,2.108,1,0
2019-04-30 00:00:00+00:00,AT,3.768,0.0,0.0,2019-04-30,10.341215,16.567,3.435,1,0



Load data


,country,load_actual,day,hour,week_of_year,month,year,is_weekend,is_holiday
utc_timestamp,,,,,,,,,
2015-01-01 00:00:00+00:00,DE,41230.04,2015-01-01,0,1,1,2015,0,1
2015-01-01 00:15:00+00:00,DE,40809.76,2015-01-01,0,1,1,2015,0,1


...


nrow weather data: 227526 nrow load data: 910086


227526

910104

910086

910086

Currently, a few rows unexpected additional weather data exists

Downsample Actuals from 15 min -> 1 hour

In [130]:
load_downsample = pd.DataFrame({})

# Downsample by country, merge numeric and non-numeric
for country in load_dt['country'].unique():
    
    # Country level data
    country_dt = load_dt.loc[load_dt['country'] == country]

    # Seperate numeric and non-numeric data
    non_numeric = country_dt.select_dtypes(exclude=['number']).columns
    time_country = country_dt[non_numeric].resample('h').first()

    # Combined
    country_dt = (load_dt.loc[load_dt['country'] == country]
                         .resample('h', level=0, group_keys=True)
                         .mean(numeric_only=True)
                         .merge(time_country, how='left', on=['utc_timestamp'])
                         [['day', 'month', 'hour', 'country', 'load_actual', 'is_weekend', 'is_holiday']])
    
    load_downsample = pd.concat([load_downsample, country_dt])

# View data
load_downsample.head()

# Ensure Weather and Load data have same number of rows
print(f'\n Weather data nrows: {len(wthr_dt)} Load data nrows: {len(load_downsample)}')

,day,month,hour,country,load_actual,is_weekend,is_holiday
utc_timestamp,,,,,,,
2015-01-01 00:00:00+00:00,2015-01-01,1.0,0.0,DE,40665.1075,0.0,1.0
2015-01-01 01:00:00+00:00,2015-01-01,1.0,1.0,DE,39509.8525,0.0,1.0
2015-01-01 02:00:00+00:00,2015-01-01,1.0,2.0,DE,38537.0625,0.0,1.0
2015-01-01 03:00:00+00:00,2015-01-01,1.0,3.0,DE,38152.6500,0.0,1.0
2015-01-01 04:00:00+00:00,2015-01-01,1.0,4.0,DE,37927.0550,0.0,1.0



 Weather data nrows: 227526 Load data nrows: 227526


Upsample Temperature 1 hour -> 15 min, Interpolate

In [159]:
# Upsample weather
# country_dt_upsample = wthr_dt.resample('15min').asfreq()
# country_dt_upsample.head()

ffill_vars = ['country', 'day', 'mean_temp', 'max_temp', 'min_temp', 'hdd', 'cdd']
interp_vars = ['temperature', 'radi_direct', 'radi_diffuse']
wthr_upsample = pd.DataFrame({})

# wthr_dt = wthr_dt.drop_duplicates()

for country in wthr_dt['country'].unique():
    country_upsample = wthr_dt.loc[lambda x: x.country == country].resample('15min').asfreq()
    # interpolate ts variables, and ffill categorical variables
    interped = country_upsample[interp_vars].interpolate()
    ffilled = country_upsample[ffill_vars].ffill() # consider including day country

    # merge
    upsampled = interped.merge(ffilled, how='left', on='utc_timestamp')

    wthr_upsample = pd.concat([wthr_upsample, upsampled])

wthr_upsample.head(5)

print(f"Ensure correct counts for upsampled dataframes:\nwthr_dt - {len(wthr_dt)}, country_upsample - {len(country_upsample)}, wthr_upsample - {len(wthr_upsample)}\n\nEnsure 4 * len(wthr_dt) == len(wthr_upsample): wthr_dt - {4 * len(wthr_dt)}, wthr_upsample - {len(wthr_upsample)}")

print(f'\nDifference due to set_index')

,temperature,radi_direct,radi_diffuse,country,day,mean_temp,max_temp,min_temp,hdd,cdd
utc_timestamp,,,,,,,,,,
2015-01-01 00:00:00+00:00,-1.983,0.0,0.0,LU,2015-01-01,-1.119535,3.298,-8.399,1.0,0.0
2015-01-01 00:15:00+00:00,-2.008,0.0,0.0,LU,2015-01-01,-1.119535,3.298,-8.399,1.0,0.0
2015-01-01 00:30:00+00:00,-2.033,0.0,0.0,LU,2015-01-01,-1.119535,3.298,-8.399,1.0,0.0
2015-01-01 00:45:00+00:00,-2.058,0.0,0.0,LU,2015-01-01,-1.119535,3.298,-8.399,1.0,0.0
2015-01-01 01:00:00+00:00,-2.083,0.0,0.0,LU,2015-01-01,-1.119535,3.298,-8.399,1.0,0.0


Ensure correct counts for upsampled dataframes:
wthr_dt - 227526, country_upsample - 151681, wthr_upsample - 910086

Ensure 4 * len(wthr_dt) == len(wthr_upsample): wthr_dt - 910104, wthr_upsample - 910086

Difference due to set_index


In [ ]:
test = wthr_dt.reset_index()
test.head()


,utc_timestamp,country,temperature,radi_direct,radi_diffuse,day,mean_temp,max_temp,min_temp,hdd,cdd
0,2015-01-01 00:00:00+00:00,LU,-1.983,0.0,0.0,2015-01-01,-1.119535,3.298,-8.399,1,0
1,2015-01-01 01:00:00+00:00,LU,-2.083,0.0,0.0,2015-01-01,-1.119535,3.298,-8.399,1,0
2,2015-01-01 02:00:00+00:00,LU,-2.075,0.0,0.0,2015-01-01,-1.119535,3.298,-8.399,1,0
3,2015-01-01 03:00:00+00:00,LU,-2.234,0.0,0.0,2015-01-01,-1.119535,3.298,-8.399,1,0
4,2015-01-01 04:00:00+00:00,LU,-2.323,0.0,0.0,2015-01-01,-1.119535,3.298,-8.399,1,0


227526

227526

In [145]:
len(wthr_dt)
len(wthr_dt.drop_duplicates())
print(f'\n')
len(load_dt)
len(load_dt.drop_duplicates())

227526

227339

910086

901800

Merge datasets

In [ ]:
load_wthr_down = load_downsample.merge(wthr_dt, how='left', on=['utc_timestamp', 'country', 'day'])
load_wthr_down.head()

# # Upsample
# load_wthr_up = wthr_upsample.merge(load_dt, how='left', on=['utc_timestamp', 'country', 'day'])
# load_wthr_up.head()

,day,month,hour,country,load_actual,is_weekend,is_holiday,temperature,radiation_direct_horizontal,radiation_diffuse_horizontal,mean_temp,max_temp,min_temp,hdd,cdd
utc_timestamp,,,,,,,,,,,,,,,
2015-01-01 00:00:00+00:00,2015-01-01,1.0,0.0,DE,40665.1075,0.0,1.0,-0.981,0.0,0.0,-1.119535,3.298,-8.399,1,0
2015-01-01 01:00:00+00:00,2015-01-01,1.0,1.0,DE,39509.8525,0.0,1.0,-1.035,0.0,0.0,-1.119535,3.298,-8.399,1,0
2015-01-01 02:00:00+00:00,2015-01-01,1.0,2.0,DE,38537.0625,0.0,1.0,-1.109,0.0,0.0,-1.119535,3.298,-8.399,1,0
2015-01-01 03:00:00+00:00,2015-01-01,1.0,3.0,DE,38152.6500,0.0,1.0,-1.166,0.0,0.0,-1.119535,3.298,-8.399,1,0
2015-01-01 04:00:00+00:00,2015-01-01,1.0,4.0,DE,37927.0550,0.0,1.0,-1.226,0.0,0.0,-1.119535,3.298,-8.399,1,0


HDD and CDD Load Shape clusters

In [ ]:
# Prep dataframe 
day_pivot_hourly = dd_hourly_pivot(load_wthr)

# Transform and normalize data for time series plots
avg_hourly = (dd_plot_norm_avg_hourly(day_pivot_hourly, ['country', 'hdd', 'cdd'])
              .reset_index()
              .assign(hour = lambda x: x['hour'].astype(int)))
print(f'\nPivoted, normalized, hourly means per country')
avg_hourly.head(4)
# # plot overlapping load shapes
avg_hourly_HDD = avg_hourly.loc[avg_hourly['hdd'] == 1]
avg_hourly_CDD = avg_hourly.loc[avg_hourly['cdd'] == 1]
print(f'Ensure HDD and CDD dataframes are half the avg_hourly length {len(avg_hourly)}:\nHDD only: {len(avg_hourly_HDD)}, CDD only: {len(avg_hourly_CDD)}')

avg_hourly_HDD.head(2)
avg_hourly_CDD.head(2)

In [ ]:
# HDD
multi_lineplots(avg_hourly_HDD, group='country')
# CDD
multi_lineplots(avg_hourly_CDD, group='country')

Observations for the above plots:
- Unlike residential data, a large amount of energy usage in mornings on HDD and CDD days, likely due to commercial usage. 
- CDD show much less usage in the afternoon

Create Lags and Rolling Means

In [ ]:
# Downsampled Data
#load_wthr.head()


### functionalize for downsampled and up sampled data (df), list of variables, number hours
# Add lags and rolling means
to_transform = ['load_actual', 'temperature']
hours = [24,48,7*24,14*24] # 1,2,7, and 14 days

lags_and_rolling_means(load_wthr, 
                       list_vars=to_transform, 
                       group='country',
                       list_hours=hours)

# View variables (tail to ensure non-NaN values)
#load_wthr.iloc[:,15:].tail()

# Upsampled Data

,day,month,hour,country,load_actual,is_weekend,is_holiday,temperature,radiation_direct_horizontal,radiation_diffuse_horizontal,...,load_actual_lag14_days,load_actual_roll14_days,temperature_lag1_days,temperature_roll1_days,temperature_lag2_days,temperature_roll2_days,temperature_lag7_days,temperature_roll7_days,temperature_lag14_days,temperature_roll14_days
utc_timestamp,,,,,,,,,,,,,,,,,,,,,
2015-01-01 00:00:00+00:00,2015-01-01,1.0,0.0,DE,40665.1075,0.0,1.0,-0.981,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-01 01:00:00+00:00,2015-01-01,1.0,1.0,DE,39509.8525,0.0,1.0,-1.035,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-01 02:00:00+00:00,2015-01-01,1.0,2.0,DE,38537.0625,0.0,1.0,-1.109,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-01 03:00:00+00:00,2015-01-01,1.0,3.0,DE,38152.6500,0.0,1.0,-1.166,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-01 04:00:00+00:00,2015-01-01,1.0,4.0,DE,37927.0550,0.0,1.0,-1.226,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-04-29 20:00:00+00:00,2019-04-29,4.0,20.0,LU,357.2425,0.0,0.0,7.499,0.0,0.0,...,548.4750,9320.573650,5.048,10.337958,5.403,9.812917,13.404,12.097280,7.681,12.732976
2019-04-29 21:00:00+00:00,2019-04-29,4.0,21.0,LU,382.9975,0.0,0.0,7.008,0.0,0.0,...,534.0250,9324.330703,4.613,10.406500,5.158,9.833292,12.533,12.075149,7.124,12.736057
2019-04-29 22:00:00+00:00,2019-04-29,4.0,22.0,LU,441.9300,0.0,0.0,6.795,0.0,0.0,...,544.1350,9328.301321,4.333,10.486708,4.813,9.856000,11.693,12.057196,6.607,12.740482


In [51]:
var = 'load_actual'
hour = 24
load_wthr['test'] = load_wthr.groupby('country', as_index=False)[var].rolling(hour).mean()[['load_actual']]

load_wthr.iloc[:,13:].head()

,hdd,cdd,test
utc_timestamp,,,
2015-01-01 00:00:00+00:00,1,0,NaN
2015-01-01 01:00:00+00:00,1,0,NaN
2015-01-01 02:00:00+00:00,1,0,NaN
2015-01-01 03:00:00+00:00,1,0,NaN
2015-01-01 04:00:00+00:00,1,0,NaN


In [ ]:
def mape_objective(y_pred, dtrain):
    y_true = dtrain.get_label()
    epsilon = 1e-6
    grad = 2 * (y_pred - y_true) / (np.abs(y_true) + epsilon)
    hess = 2 / (np.abs(y_true) + epsilon)
    return grad, hess
def mape_metric(y_true, y_pred):
    epsilon = 1e-6
    return np.mean(np.abs((y_true - y_pred) / (np.abs(y_true) + epsilon))) * 100

In [ ]:
### Goal: Simple, incremental build of LightGBM Forecasting model











# Decision - to create seperate forecasts per country? (Of course)
# Giant for loop 
# Save results to dictionary
# results = {}
# horizon = 7 # Days

# Levels to loop through:
    # Downsampled Data, Upsampled Data
        # Countries 

# for country in load_wthr['country'].unique():
    # filter data to country:
    # train/test splits, param search, map and mae
    # save info in results for that country
        # key: country, items: mape/mae, best model params, other stuff


[I 2025-02-19 09:34:36,359] A new study created in memory with name: no-name-d5f9d136-6756-4995-8da0-dae6a8d2e5d6
[W 2025-02-19 09:34:36,361] Trial 0 failed with parameters: {} because of the following error: NameError("name 'your_data_frame' is not defined").
Traceback (most recent call last):
  File "c:\Users\WulfN\Python Projects\venv\Lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\WulfN\AppData\Local\Temp\ipykernel_4912\3431307969.py", line 67, in <lambda>
    study.optimize(lambda trial: objective(trial, your_data_frame), n_trials=50)
                                                  ^^^^^^^^^^^^^^^
NameError: name 'your_data_frame' is not defined
[W 2025-02-19 09:34:36,376] Trial 0 failed with value None.


NameError: name 'your_data_frame' is not defined

: 

In [ ]:
##